In [ ]:
# Import general packages
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import pandas as pd

In [ ]:
# Import files
import firecurve as fc
import running as rn
import makefiles as mf
import thermalrestart as tr
from Analytical_solution_Dirichlet import analytical_dirichlet
import Post_Processor as pp

In [ ]:
# ---------------- Material Inputs ----------------
e0 = 0.02            # Eccentricity [m]

rho = 420            # Density (Including moisture) [kg/m^3]
mu = 0.3             # Poisson ratio [-]

E = 11.5e9           # Modulus of Elasticity [Pa]
f_c = 24.0e6            # Compressive strentgh [Pa]
f_t = 19.2e6          # Tensile strength [Pa]

# E = 12.8e9    
# f_c = 40.4e6
# f_t = 26.4e6


w = 12               # Moisture content [%]

h_ch = 35            # Conection coefficient heating [W/m^2 K]
h_cc = 4             # Convection coefficient cooling [W/m^2 K]
eps = 0.8            # Relative emissivity [-]
struc_material_name = 'WOODPRBWE'
struc_material_params = (E, mu, f_c, f_t, 0.5, 1)
# struc_material_name = 'WOODEC5'
# struc_material_params = (E, mu, f_c, f_t, 1)
# ---------------------------------------------
# ---------------- Structural Inputs ----------
B = 0.28          # Column width [m]
l = 3.65          # Column length [m]
F = 322_000        # Applied force [N]
# ---------------------------------------------
# ---------------- Fire Inputs ----------------
T0     = 20.0     # ambient [°C]
t_h    = 150     # heating duration to peak [min]
T_max  = round(T0 + 1325 * fc.phi(t_h/60), 2)   # peak gas temperature at t = t_h [°C] (<=1325). Tau is equal to 1. (ISO heating)
# T_max  = 1150   # peak gas temperature at t = t_h [°C] (<=1325).
r_cool = 10.4      # linear cooling rate [°C/min]
t_end  = round(analytical_dirichlet(B, rho, T_max, t_h, r_cool, c=1530, k=0.12)[0]*1.3, 2)   # Initial end time of the curve [min] (>= t_h), analytically determined with dirichlet boundary condition of T_g(t) times 1.3.
dt     = 0.5      # numerical time step of fire curve heating phase [min]
# --------------------------------------------------
# ---------------- Numerical Inputs ----------------
# fine_size = 0.005         # Mesh size of the thermal analysis
fine_size = mf.compute_fine_size_closest(B, 0.005)  # Determination of cross-sectional mesh size for the thermal analysis
n_elements, mesh_size = mf.compute_mesh_and_nelements(l, 0.25) # Determination of the beam element mesh size for the structural analysis
# n_elements = 21    # Mesh size of structural analysis
# ---------------------------------------------
print(n_elements)
print(fine_size)
print(t_end)

In [ ]:
q_dist = B*B*rho*10    # Calculation of self weight of the column
M0 = e0 * F            # Calculation of the intial moment due to forcing and eccentricity
t_end_guess = t_end

In [ ]:
# Setting up working directory and filenames
workdir =       # local computer working directory
name_firecurve = f'fr_233.fct'
name_thermalfile = f'thermo_233.IN'
name_mechfile = f'COLUMN_233.IN'

In [ ]:
# Creation of Parametric Firecurve
t_all, T_all, tau = fc.build_fire_curve(T0=T0, T_max=T_max, t_h=t_h, r_cool=r_cool, t_end=t_end, dt=dt, plot_curve=True)
# H = fc.area_firecurve(T0, T_max, t_h, r_cool, t_end, tau)
# print(f"Exact area above ambient up to t_end: {H:.2f} °C·min  ({H/60:.2f} °C·h)")
outfile = fc.save_fct(t_all, T_all, filename=name_firecurve, out_dir=workdir)

In [ ]:
# Initialize the thermal module to use 
# tau = 11.3
# T_start = 100
# T_end = 120
# therm_material_name = 'USER1'  # Use HOPKINS thermal model, with Cachim and Franssen
# therm_material_params = (rho, w, T_start, T_end, h_ch, h_cc, eps, 1, tau)   # Material reversibility -1  / Mateterial irreversibility +1
therm_material_name = 'WOODEC5'
therm_material_params = (rho, w, h_ch, h_cc, eps, 4, 0, 0, 1)

In [ ]:
# # Importing experimental fire curve data when needed 
# data = np.loadtxt('Natural_fire_test_data/TEST17/TF_17.fct')
# times = data[:, 0]/60
# Temps = data[:, 1]
# plt.plot(times, Temps)
# plt.plot(t_all, T_all)
# plt.xlim(0, 200)

# outfile = fc.save_fct(times, Temps, filename=name_firecurve, out_dir=workdir)
# print(tau)
# t_end = times[-1]
# t_end

In [ ]:
# # Walking flux model initialisation (See other Notebook 'SingleRun_TESTFLUX.ipynb')

In [ ]:
# Thermal Analysis
nfiber = mf.make_cross_section_SYM(workdir=workdir, filename=name_thermalfile, B=B, fine_size=fine_size, frontier_name=name_firecurve, T_initial=20.0, comeback=0.001, precision=1e-3, dt0=7.5, t_final=t_end*60, dtmax=30.0, timeprint=60.0, material_name=therm_material_name, material_params=therm_material_params)
inp = Path(workdir) / Path(name_thermalfile).stem
rn.run_safir(input_file=inp, env_var="SAFIREXE", workdir=workdir)
restart = tr.thermal_restart(workdir, name_thermalfile, B, fine_size, extend_step_s=1800)
for r in range(1, 15):   # Arbitrary choice of 15 iterations when restart algorithm is started
    if restart['runs'] > 1:
        t_end = restart['last_time']/60
        workdir = Path(workdir) / f'Batch0{r}'
        t_all, T_all, tau = fc.build_fire_curve(T0=T0, T_max=T_max, t_h=t_h, r_cool=r_cool, t_end=t_end, dt=dt, plot_curve=False)
        H = fc.area_firecurve(T0, T_max, t_h, r_cool, t_end, tau)
        outfile = fc.save_fct(t_all, T_all, filename=name_firecurve, out_dir=workdir)
        nfiber = mf.make_cross_section_SYM(workdir=workdir, filename=name_thermalfile, B=B, fine_size=fine_size, frontier_name=name_firecurve, T_initial=20.0, comeback=0.01, precision=1e-3, dt0=7.5, t_final=t_end*60, dtmax=30.0, timeprint=60.0, material_name=therm_material_name, material_params=therm_material_params)
        inp = Path(workdir) / Path(name_thermalfile).stem
        rn.run_safir(input_file=inp, env_var="SAFIREXE", workdir=workdir)
        restart = tr.thermal_restart(workdir, name_thermalfile, B, fine_size)
    else:
        break

In [ ]:
# Structural Analysis
tem_file = f"{Path(name_thermalfile).stem}.TEM"
mf.make_column(workdir=workdir, filename=name_mechfile, length=l, tem_file=tem_file, nfiber=nfiber, elements=n_elements, ng=3 ,comeback=1e-09,  F_function='FLOAD', nodeload1=(0.0, 0.0, -M0), nodeload2=(0.0, -F, M0), q_dist=-q_dist, t_final=t_end*60, timeprint=60.0 ,material_name=struc_material_name, material_params=struc_material_params, stresses=True)
inp_2 = Path(workdir) / Path(name_mechfile).stem
rn.run_safir(input_file=inp_2, env_var="SAFIREXE", workdir=workdir)

In [ ]:
# Post-processing steps
failure_time, failure = pp.postprocess_failure(workdir, f'{Path(name_mechfile).stem}.OUT', t_end)
time_thermo = pp.get_simulation_time(workdir, f'{Path(name_thermalfile).stem}.OUT')
time_mech = pp.get_simulation_time(workdir, f'{Path(name_mechfile).stem}.OUT')
time_tot = pp.sum_times([time_thermo, time_mech])
pp.make_description(workdir, 'AA_Description.txt', time_thermo, time_mech, time_tot, e0, rho, E, mu, f_c, f_t, w, h_ch, h_cc, eps, B, l, F, T0, t_h, T_max, r_cool, t_end, dt, fine_size, n_elements, failure, failure_time)

In [ ]:
# nodes = [] 
# names_tcs = ['10 mm', '20 mm', '30 mm', '40 mm', '50 mm', '60 mm', '70 mm', '80 mm']
# thermo_couples = pp.get_temperatures(workdir, f'{Path(name_thermalfile).stem}.OUT', nodes, column_names=names_tcs)

In [ ]:
failure_time

In [ ]:
failure